# 猎鹰H1的不塞培训

该笔记本是由TII Falcon团队撰写的。
有关Falcon H1系列模型的更多详细信息：
1。xx_markDown_link_xx
2。xx_markDown_link_xx
3。xx_markDown_link_xx
4。xx_markDown_link_xx

要运行此操作，请按“*Runtime*”，然后按“*运行*” ** free ** tesla Tesla T4 Google COLAB实例！
<div class =“ Align-Center”>

<a href =“ https://unsloth.ai/”> <img src =“ https://github.com/unslothai/unslothai/unsloth/raw/raw/main/main/mains/unsloth%20new%20new%20logo.png.png”
<a href =“ https://discord.gg/unsloth”> <img src =“ https://github.com/unslothai/unslothai/unsloth/raw/main/main/main/images/images/discord button.png button.png
<a href =“ https://docs.unsloth.ai/”> <img src =“ https://github.com/unslothai/unslothai/unsloth/unsloth/main/main/main/mains/images/documentation%20green%20green%20breen%20button.png?png?raw=raw=true width width” <i>在<a href =“ https://github.com/unslothai/unsloth”> github </a> </i>⭐中
</div>

要在您自己的计算机上安装不塞，请按照我们的github页面上的安装说明[here](https://docs.unsloth.ai/get-started/installing-+-updating)上的安装说明。

您将学习如何做[data prep](#Data)，如何[train](#Train)，如何[run the model](#Inference)，＆xx_markDown_link_xx

### 安装

In [ ]:
%%capture
# 安装Unsploth，Xformers（Flash注意力）和所有其他软件包！
!pip install unsloth
# 获取最新的不塞
!pip uninstall unsloth -y

In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git@main"

In [ ]:
!pip install vllm

In [ ]:
!pip install --force-reinstall git+https://github.com/huggingface/transformers.git 

In [ ]:
!pip install --no-build-isolation git+https://github.com/Dao-AILab/causal-conv1d.git@main


In [ ]:
!pip install --no-build-isolation git+https://github.com/state-spaces/mamba.git@main

In [ ]:
!pip install git+https://github.com/unslothai/unsloth-zoo.git

In [ ]:
import unsloth
from unsloth import FastLanguageModel
import torch
import os
os.environ['TRITON_JIT_DISABLE_OPT'] = '1' # 可能是最关键的变化

max_seq_length = 2048 # 选择任何！我们在内部自动支持绳索扩展！
dtype = None # 没有自动检测。 Tesla T4，V100，Bfloat16的float16 for Ampere+
load_in_4bit = True # 使用4位量化来减少内存使用量。可以是错误的。

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "tiiuae/Falcon-H1-0.5B-Instruct", # 从https://huggingface.co/collections/tiiuae/falcon-h1-6819f2795bc406da60fab8df中选择任何型号
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

In [ ]:
# 配置PEFT模型
model = FastLanguageModel.get_peft_model(
    model,
    r=64,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"], # MAMBA OUT_PROJ和CONV1D层不应在此处包括
    lora_alpha= 32,
    lora_dropout= 0.1,
    use_gradient_checkpointing=False,
    random_state=3407,
)

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

# ＃＃ 操作说明：
{}

# ＃＃ 输入：
{}

# ＃＃ 回复：
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # 必须添加eos_token，否则您这一代将永远持续下去！
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    packing = False, # 可以使短序列更快地训练5倍。
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 8,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

### 显示当前的内存统计

In [ ]:

gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

# 训练

In [ ]:
import os
os.environ["TRITON_DISABLE_LINE_INFO"] = "1"
trainer_stats = trainer.train()

### 显示最终记忆和时间统计

In [ ]:
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")